In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/my-vector-index/index.faiss
/kaggle/input/my-vector-index/index.pkl


In [2]:
# Core requirements
!pip install -q --upgrade \
    torch torchvision torchaudio \
    transformers accelerate sentence-transformers \
    langchain langchain-community faiss-cpu \
    pypdf pymupdf

# For GPU support (uncomment if needed)
# !pip install -q faiss-gpu==1.7.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 821.2/821.2 MB 2.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 393.1/393.1 MB 2.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 75.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 61.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.7/897.7 kB 44.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 571.0/571.0 MB 3.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.2/200.2 MB 8.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 53.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 30.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.2/158.2 MB 10.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
pip install langchain_huggingface

Note: you may need to restart the kernel to use updated packages.


In [4]:
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer
from huggingface_hub import login
import torch

# Memory optimization
torch.cuda.empty_cache()
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

def load_model_from_hub(repo_name: str):
    tokenizer = AutoTokenizer.from_pretrained(repo_name)
    model = AutoModelForCausalLM.from_pretrained(
        repo_name,
        device_map="auto",
        torch_dtype=torch.float16,
        trust_remote_code=True
    )
    return pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        max_new_tokens=512,
        temperature=0.7
    )

# Initialize
login(token="your_token")

2025-08-03 16:15:52.645972: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1754237752.815067      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1754237752.868267      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [6]:
print("STEP 3: Defining the Question-Answering class...")

from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain_huggingface import HuggingFacePipeline
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings

class RobustGateQA:
    def __init__(self, pipe, vector_store):
        self.llm = HuggingFacePipeline(pipeline=pipe)
        self.vector_store = vector_store
        
        # This prompt template is designed to act like a tutor, asking the AI
        # to analyze the context and provide a clear, evidence-based answer.
        self.qa_prompt = PromptTemplate(
            input_variables=["context", "question"],
            template="""
            Analyze the following excerpt from a GATE DSAI study document to answer the user's question.
            
            Excerpt:
            {context}
            
            Question: {question}
            
            Based on the excerpt, provide a clear, concise, and technically accurate answer.
            Then, quote the specific sentence or formula from the excerpt that serves as direct evidence for your answer.
            
            Format your response as:
            ANSWER: [Your clear and direct answer]
            EVIDENCE: [The exact quote or evidence from the excerpt]
            """
        )
        
        # Configure the QA chain. We use a retriever to find the most relevant
        # document chunks based on the user's question.
        self.qa_chain = RetrievalQA.from_chain_type(
            llm=self.llm,
            chain_type="stuff",
            retriever=self.vector_store.as_retriever(
                search_kwargs={
                    "k": 1, # Find the single best matching chunk
                    "score_threshold": 0.65 # Set a relevance threshold
                }
            ),
            chain_type_kwargs={"prompt": self.qa_prompt},
            return_source_documents=True
        )
    
    def ask(self, question: str):
        """Asks a question to the QA chain and processes the result."""
        torch.cuda.empty_cache()
        try:
            result = self.qa_chain.invoke({"query": question})
            
            response = {
                "answer": "I could not find a relevant answer in your documents.",
                "evidence": "No direct evidence was found.",
                "source": "N/A",
                "document_preview": ""
            }
            
            if result["source_documents"]:
                doc = result["source_documents"][0]
                response.update({
                    "source": doc.metadata.get("source", "Unknown"),
                    "document_preview": doc.page_content[:400] + "..."
                })
                
                # Parse the LLM's structured response
                answer_lines = result["result"].split('\n')
                for line in answer_lines:
                    line = line.strip()
                    if line.startswith("ANSWER:"):
                        response["answer"] = line.replace("ANSWER:", "").strip()
                    elif line.startswith("EVIDENCE:"):
                        response["evidence"] = line.replace("EVIDENCE:", "").strip()
            
            return response
            
        except Exception as e:
            return {"error": str(e)}

print("Question-Answering class defined.\n")

STEP 3: Defining the Question-Answering class...
Question-Answering class defined.



In [8]:
pipe = load_model_from_hub("sravan837/mistral-7b-medical-qa")
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
print("  -> Loading the vector index...")
# This is the embedding model used to understand your documents and questions.
embeddings = HuggingFaceEmbeddings(
    model_name="BAAI/bge-m3",
    model_kwargs={"device": "cuda" if torch.cuda.is_available() else "cpu"}
)

# Load the pre-built index from your Kaggle dataset.
vector_store_path = "/kaggle/input/my-vector-index"
try:
    vector_store = FAISS.load_local(
        vector_store_path,
        embeddings,
        allow_dangerous_deserialization=True
    )
    print("  -> Vector index loaded successfully.")
    qa_system = RobustGateQA(pipe, vector_store)
    print("QA system is ready.\n")
except Exception as e:
    print(f"  -> ERROR: Failed to load vector store from '{vector_store_path}'. {e}")
    qa_system = None


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Device set to use cuda:1


  -> Loading the vector index...
  -> Vector index loaded successfully.
QA system is ready.



In [9]:
if qa_system:
    # You can add any questions you want to study here.
    questions = [
        "What is a Convolutional Neural Network (CNN)?",
        "Explain the concept of integration by parts.",
        "What is the primary key in a relational database?",
    ]
    
    print("--- GATE DSAI TUTOR IS READY ---")
    
    for i, question in enumerate(questions):
        print(f"\n=========================================================")
        print(f" TUTORING SESSION {i+1} ")
        print(f"=========================================================")
        print(f"\n🤔 Your Question: \"{question}\"")
        
        response = qa_system.ask(question)
        
        if "error" in response:
            print(f"\n❌ An error occurred: {response['error']}")
        else:
            print("\nHere's what I found in your study materials:")
            print("---------------------------------------------------------")
            print(f"✅ ANSWER: {response['answer']}")
            print("---------------------------------------------------------")
            print(f"🔍 EVIDENCE: \"{response['evidence']}\"")
            print("---------------------------------------------------------")
            print(f"📂 This information was found in the document: {response['source']}")
            print("\nHere is a preview of the section I used for context:")
            print(".........................................................")
            print(response["document_preview"])
            print(".........................................................")
else:
    print("\n--- Could not start the tutoring session because the QA system failed to initialize. ---")


--- GATE DSAI TUTOR IS READY ---

 TUTORING SESSION 1 

🤔 Your Question: "What is a Convolutional Neural Network (CNN)?"


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `XLMRobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `XLMRobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



Here's what I found in your study materials:
---------------------------------------------------------
✅ ANSWER: Convolutional Neural Networks (CNNs) are a type of deep neural network that use cross-correlations instead of fully connected layers to recognize and classify images.
---------------------------------------------------------
🔍 EVIDENCE: ""Convolutional layers actually use cross-correlations, which are very similar to convolutions (see http://goo.gl/HAfxXd for more details).""
---------------------------------------------------------
📂 This information was found in the document: /kaggle/input/gate-dsai-llm/Gate_DSAI/Machine-Learning/Hands On Machine Learning with Scikit Learn and TensorFlow.pdf

Here is a preview of the section I used for context:
.........................................................
6 A convolution is a mathematical operation that slides one function over another and measures the integral of
their pointwise multiplication. It has deep connections with t

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `XLMRobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



Here's what I found in your study materials:
---------------------------------------------------------
✅ ANSWER: I could not find a relevant answer in your documents.
---------------------------------------------------------
🔍 EVIDENCE: "No direct evidence was found."
---------------------------------------------------------
📂 This information was found in the document: N/A

Here is a preview of the section I used for context:
.........................................................

.........................................................

 TUTORING SESSION 3 

🤔 Your Question: "What is the primary key in a relational database?"

Here's what I found in your study materials:
---------------------------------------------------------
✅ ANSWER: I could not find a relevant answer in your documents.
---------------------------------------------------------
🔍 EVIDENCE: "No direct evidence was found."
---------------------------------------------------------
📂 This information was found in